We'll at least start out with the given guys before feature engineering.

In [23]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import RFECV #was hoping for this without the reverse, but I don't see it. Oh well when
from sklearn.model_selection import cross_val_score
#it's just 8; well, it'll be more hectic momentarily...

# from sklearn.metrics import make_scorer
from sklearn.metrics import root_mean_squared_log_error #Phew this on is on there... I mean find if we'd have to square,
#but rather not have to write the formula.

# from sklearn.metrics import make_scorer

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR

In [3]:
df = pd.read_csv('../data/cleaned_train.csv')
print(df.shape)
df.head()

(90609, 9)


,sex,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [4]:
#I honestly forgot to earlier, although I do like the data is preserved: At least at the moment we also need to dummify sex
#to get our lovely sklearn methods to work:

df = pd.get_dummies(df, columns=['sex'], dtype=int, drop_first=True)
df.head()  #Note that the baseline, ie 0s in the rest, will be indicativ of female abalones.

,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,rings,sex_I,sex_M
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,0,0
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,0,0
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,1,0
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,0,1
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,1,0


In [5]:
X = df.drop('rings', axis=1)
y = df['rings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=26) #Relatively large test size as we
#have so much data to work with...
#Re. 26 - I like that number. Discussion not for now

mms = MinMaxScaler() #Temp. removed to have an easier time seeing which feature names were used.
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

Given that this is somewhat of a professional environment, as opposed to my boot camp I just finished, I won't bother to explain all of my thoughts. However, note that I'll reuse this and similar lines of code until finding desired results.

In [6]:
# model_defaults.get_feature_names_out()

#Hmm, quite interesting that we chose all the weights.... ie I'd understand if one, but
#apparetly the relative discremencies upon focusing on less and less of the alabone, quite literally, is that useful.

#Also, the distinguishment between female is somehwat negligible, despite hte many difference we obsered... however from
#female to on-binary remains significant.

In [34]:
#Oh, just remembered we're supposed to use MSLE. At th least we have a benchmark of what's good 'objectively'

lr = LinearRegression()
rfe = RFECV(lr, step=1, cv=5,
                       min_features_to_select=3)

model_defaults = rfe.fit(X_train, y_train)

preds_model_defaults = rfe.predict(X_test)#And we're somehow getting some negative predictions.... note that againt his is
#without any feature engineering.... Regardless, we'll just brute force it. Unsure if we have a better wayt han the following:

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0
print(preds_model_defaults[preds_model_defaults<0])

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
print(model_defaults.cv_results_['mean_test_score'])
root_mean_squared_log_error(y_test, preds_model_defaults)

[]
0.6062234959016377 0.6034787674182922
[0.59003897 0.59367525 0.59568496 0.59954274 0.59970396 0.60580828
 0.60580744]


0.16508650876858538

In [36]:
#A little tempted to play around with the defaults, but eh...consideing that we likely need to engineer some things...

rf = RandomForestRegressor(random_state=26)

##Seems a bit redundant to use both here...
# rfe = RFECV(rf, step=1, cv=5, 
#                        min_features_to_select=3)

model_defaults = rf.fit(X_train, y_train)

preds_model_defaults = rf.predict(X_test)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
# print(cross_val_score(rf, 
#                       X_train, y_train, cv = 5).mean()) #Yeah,c ommenting this guy out... taking too long plus a bit redundant
# #Given that random forests are already aggregativing everything
root_mean_squared_log_error(y_test, preds_model_defaults)

#oooooh yesss this is quite juicy... quite juicy abalane 

0.9504620112507841 0.6557858728716266
0.6445375107390473


0.15345037771109488

In [ ]:
#Some notes in case I need them

In [ ]:
pipe = Pipeline([
    ('cvec', CountVectorizer(lowercase=False)),
    ('bc', BaggingClassifier(random_state=26))
])
pipe_params = {'cvec__min_df' : [0.0]
               ,'cvec__max_df' : [.95]
               ,'cvec__ngram_range' : [(1,2)]
               ,'cvec__max_features' : [1800, 1750, 1700]
               ,'cvec__stop_words' : ['english']
              }
gs = GridSearchCV(pipe,
                  param_grid=pipe_params,
                  cv=5)

gs.fit(X_train, y_train)

In [ ]:
print(gs.score(X_train, y_train), gs.score(X_test, y_test))
print(gs.best_score_)
print(gs.best_params_)
#Oh wow, this did a lot 